In [1]:
# https://pmc.ncbi.nlm.nih.gov/articles/PMC7779747/
# https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE279781
# https://www.cell.com/iscience/pdf/S2589-0042(24)02796-2.pdf

In [2]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [3]:
import sys
variables = {var: sys.getsizeof(obj) / (1024**2)  # List variables with their memory size
             for var, obj in globals().items() if not var.startswith("__")}
for var, size in variables.items():
    print(f"{var}: {size:.2f} MB", end=" ")
    
print([var for var in dir() if not var.startswith('__')])
%reset -f

_ih: 0.00 MB _oh: 0.00 MB _dh: 0.00 MB In: 0.00 MB Out: 0.00 MB get_ipython: 0.00 MB exit: 0.00 MB quit: 0.00 MB open: 0.00 MB _: 0.00 MB _i: 0.00 MB _ii: 0.00 MB _iii: 0.00 MB _i1: 0.00 MB _i2: 0.00 MB _exit_code: 0.00 MB _i3: 0.00 MB sys: 0.00 MB ['In', 'Out', '_', '_dh', '_exit_code', '_i', '_i1', '_i2', '_i3', '_ih', '_ii', '_iii', '_oh', 'exit', 'get_ipython', 'open', 'quit', 'size', 'sys', 'var', 'variables']


In [4]:
import time; start = time.time()
from datetime import datetime; print(datetime.now().strftime("%B %d, %Y %H:%M:%S"))

December 30, 2024 19:33:09


In [5]:
import warnings; warnings.filterwarnings("ignore") 
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [6]:
import pandas as pd;
#pd.options.display.float_format = ' {:.3f}' .format
pd.options.display.max_columns = 200
import seaborn as sns; sns.__version__
import matplotlib.pyplot as plt

In [7]:
annotations_path  = '../../data/GSE279781/GSE279781__annotations.csv.gz'
features_path     = '../../data/GSE279781/GSE279781_features.tsv.gz'
barcodes_path     = '../../data/GSE279781/GSE279781_barcodes.tsv.gz'

In [8]:
annotations = pd.read_csv(annotations_path, header=0, sep=',', quotechar='"', on_bad_lines = "warn")
annotations.sample(5)

,barcode,is_cell,contig_id,high_confidence,length,chain,v_gene,d_gene,j_gene,c_gene,full_length,productive,fwr1,fwr1_nt,cdr1,cdr1_nt,fwr2,fwr2_nt,cdr2,cdr2_nt,fwr3,fwr3_nt,cdr3,cdr3_nt,fwr4,fwr4_nt,reads,umis,raw_clonotype_id,raw_consensus_id,exact_subclonotype_id
18731,GCTCTGTAGATGCCTT-1,True,GCTCTGTAGATGCCTT-1_contig_1,True,504,TRB,TRBV12-3,NaN,TRBJ1-2,TRBC1,True,True,DAGVIQSPRHEVTEMGQEVTLRCKPI,GATGCTGGAGTTATCCAGTCACCCCGCCATGAGGTGACAGAGATGG...,SGHNS,TCAGGCCACAACTCC,LFWYRQTMMRGLELLIY,CTTTTCTGGTACAGACAGACCATGATGCGGGGACTGGAGTTGCTCA...,FNNNVP,TTTAACAACAACGTTCCG,IDDSGMPEDRFSAKMPNASFSTLKIQPSEPRDSAVYF,ATAGATGATTCAGGGATGCCCGAGGATCGATTCTCAGCTAAGATGC...,CASSYQGRLNGYTF,TGTGCCAGCAGTTACCAGGGACGGCTAAATGGCTACACCTTC,GSGTRLTVV,GGTTCGGGGACCAGGTTAACCGTTGTAG,1836,6,clonotype10955,clonotype10955_consensus_1,1
21399,GTCAAGTGTGAACCTT-1,True,GTCAAGTGTGAACCTT-1_contig_1,True,477,TRA,TRAV21,NaN,TRAJ21,TRAC,True,True,KQEVTQIPAALSVPEGENLVLNCSFT,AAACAGGAGGTGACACAGATTCCTGCAGCTCTGAGTGTCCCAGAAG...,DSAIYN,GATAGCGCTATTTACAAC,LQWFRQDPGKGLTSLLL,CTCCAGTGGTTTAGGCAGGACCCTGGGAAAGGTCTCACATCTCTGT...,IQSSQRE,ATTCAGTCAAGTCAGAGAGAG,QTSGRLNASLDKSSGRSTLYIAASQPGDSATYL,CAAACAAGTGGAAGACTTAATGCCTCGCTGGATAAATCATCAGGAC...,CGDGNFNKFYF,TGCGGCGATGGGAACTTCAACAAATTTTACTTT,GSGTKLNVKP,GGATCTGGGACCAAACTCAATGTAAAACCAA,1560,2,clonotype2066,clonotype2066_consensus_4,1
11577,CGCCAAGCACGGTAGA-1,True,CGCCAAGCACGGTAGA-1_contig_3,True,477,TRA,TRAV12-3,NaN,TRAJ24,NaN,True,True,QKEVEQDPGPLSVPEGAIVSLNCTYS,CAGAAGGAGGTGGAGCAGGATCCTGGACCACTCAGTGTTCCAGAGG...,NSAFQY,AACAGTGCTTTTCAATAC,FMWYRQYSRKGPELLMY,TTCATGTGGTACAGACAGTATTCCAGAAAAGGCCCTGAGTTGCTGA...,TYSSGN,ACATACTCCAGTGGTAAC,KEDGRFTAQVDKSSKYISLFIRDSQPSDSATYL,AAAGAAGATGGAAGGTTTACAGCACAGGTCGATAAATCCAGCAAGT...,CAMSAPDSWGKLQF,TGTGCAATGAGCGCGCCTGACAGCTGGGGGAAATTGCAGTTT,GAGTQVVVTP,GGAGCAGGGACCCAGGTTGTGGTCACCCCAG,606,1,clonotype7536,clonotype7536_consensus_2,1
17937,GCATGCGAGTACATGA-1,True,GCATGCGAGTACATGA-1_contig_3,True,505,TRA,TRAV1-2,NaN,TRAJ7,TRAC,True,True,GQNIDQPTEMTATEGAIVQINCTYQ,GGACAAAACATTGACCAGCCCACTGAGATGACAGCTACGGAAGGTG...,TSGFNG,ACATCTGGGTTCAACGGG,LFWYQQHAGEAPTFLSY,CTGTTCTGGTACCAGCAACATGCTGGCGAAGCACCCACATTTCTGT...,NVLDGL,AATGTTCTGGATGGTTTG,EEKGRFSSFLSRSKGYSYLLLKELQMKDSASYL,GAGGAGAAAGGTCGTTTTTCTTCATTCCTTAGTCGGTCTAAAGGGT...,CAVPYGNNRLAF,TGTGCTGTGCCGTATGGGAACAACAGACTCGCTTTT,GKGNQVVVIP,GGGAAGGGGAACCAAGTGGTGGTCATACCAA,770,1,clonotype9362,clonotype9362_consensus_3,1
7723,CACCACTGTAAGTGTA-1,True,CACCACTGTAAGTGTA-1_contig_2,True,505,TRB,TRBV14,NaN,TRBJ1-1,TRBC1,True,True,EAGVTQFPSHSVIEKGQTVTLRCDPI,GAAGCTGGAGTTACTCAGTTCCCCAGCCACAGCGTAATAGAGAAGG...,SGHDN,TCTGGACATGATAAT,LYWYRRVMGKEIKFLLH,CTTTATTGGTATCGACGTGTTATGGGAAAAGAAATAAAATTTCTGT...,FVKESK,TTTGTGAAAGAGTCTAAA,QDESGMPNNRFLAERTGGTYSTLKVQPAELEDSGVYF,CAGGATGAGTCCGGTATGCCCAACAATCGATTCTTAGCTGAAAGGA...,CASPILNTEAFF,TGTGCCAGTCCCATATTGAACACTGAAGCTTTCTTT,GQGTRLTVV,GGACAAGGCACCAGACTCACAGTTGTAG,744,3,clonotype3740,clonotype3740_consensus_1,1


In [9]:
len(annotations)

29453

In [10]:
features = pd.read_csv(features_path, header=0, sep='\t', quotechar='"', on_bad_lines = "warn")
features.sample(5)

,ENSG00000243485,MIR1302-2HG,Gene Expression
27470,ENSG00000140848,CPNE2,Gene Expression
24836,ENSG00000205476,CCDC85C,Gene Expression
7146,ENSG00000285631,AC097105.1,Gene Expression
25196,ENSG00000279408,AC135068.3,Gene Expression
2523,ENSG00000286754,AL031290.1,Gene Expression


In [11]:
len(features)

36600

In [12]:
barcodes = pd.read_csv(barcodes_path, header=0, sep='\t', quotechar='"', on_bad_lines = "warn")
barcodes.sample(5)

,AAACGGGAGCAATATG-1
24735,AGAGCGACAAACTGCT-7
40906,CACACTCAGTCACGCC-11
35035,CGCCAAGGTGCTGTAT-9
85138,CCTAGCTCACGAAATA-14
138984,ACCCACTGTTAAGTAG-16


In [13]:
len(barcodes)

150627